In [ ]:
"""This section tests if PSSE and psspy work."""
import sys, os

path_pssepy = r'C:\Program Files\PTI\PSSE35\35.5\PSSPY39'  # or where else you find the psspy.pyc
sys.path.append(path_pssepy)        # Tell Python where PSSE is installed
os.environ['PATH'] += ';'+path_pssepy

path_psse = r'C:\Program Files\PTI\PSSE35\35.5\PSSBIN'  # or where else you find the psse35.exe
sys.path.append(path_psse)
os.environ['PATH'] += ';'+path_psse       # Tell PSSE where PSSE is installed

import psse35
import psspy
psspy.psseinit()

In [ ]:
"""This section creates the project folders for VRT."""
import pandas as pd
import os

# Read the project spreadsheet excel file
df_project = pd.read_excel("D:/ERCOT/AEP Dynamic/MQT/MQT.xlsx",header=None)
project_list = []
for project in df_project.values.tolist():
    num = project[0]
    name = project[1]
    project_list.append(num+' '+name)
    # path = 'D:/ERCOT/AEP Dynamic/MQT/Undone/'+num+' '+name+'/'    # Folder to output the report
    path = 'D:/DMView 3.3.1/CASEs/'+num+' '+name+'/'                # Folder to run the VRT

    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(path)

In [ ]:
"""This section defines functions to check if the project has storage and other."""

def has_ess(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    project = project.lower()
    project = project.replace("(", "")
    project = project.replace(")", "")
    if ' storage' in project or ' ess' in project or ' bess' in project or ' esc' in project:
        return True
    else:
        return False
    
def has_solar(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    project = project.lower()
    project = project.replace("(", "")
    project = project.replace(")", "")
    if ' solar' in project:
        return True
    else:
        return False
    
def has_wind(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    project = project.lower()
    project = project.replace("(", "")
    project = project.replace(")", "")
    if ' wind' in project:
        return True
    else:
        return False
    
def has_non_ess(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    if has_solar(project) or has_wind(project):
        return True
    else:
        return False
    
def is_slf(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    if has_ess(project) and has_non_ess(project):
        return True
    else:
        return False

In [ ]:
"""This section defines a function to create the ini file given the project number and name."""

def create_ini(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    vrt_path = 'D:/DMView 3.3.1/'
    
    initials = ['_lag','_lead'] # Initial settings for VRT

    if has_ess(project):    # Check if there is storage
        scenarios = ['_dis','_ch']  # Only 2 scenarios: charging and discharging
    if has_non_ess(project):  # Check if there is solar or wind
        scenarios = ['']    # Only 1 scenario
    if has_ess(project) and has_non_ess(project): # Check if it's SLF or hybrid project
        scenarios = ['','_dis','_ch','_cmb']    # 4 scenarios: Non-ESS Off, ESS On (Charging); Non-ESS Off, ESS On (Discharging); Non-ESS On, ESS Off; Non-ESS On (50%), ESS On (50%)
    
    for scenario in scenarios:
        for initial in initials:
            """Create the ini file."""
            filename = vrt_path + project + scenario + initial + '.ini' # Create the ini file for each scenarios and initial settings

            """Write the ini file."""
            with open(filename, 'w') as ini:
                ini.write('/' + project + 'for ERCOT dynamic model quality test\n')
                ini.write("/created by Arthur Li - CF Power\n\n")
                ini.write("[Build FS]\nBuild_FS_flag = 1\nPOI_PF = ")
                if initial == '_lag':
                    ini.write('0.95 /lagging \n')   # Create ini for lagging
                else:
                    ini.write('-0.95 /leading \n')  # Create ini for leading

                ini.write('\n[Input files]\ninput_path = CASEs\\'+project+'\n')
                ini.write('unconv_casefile ='+project+scenario+'.sav\nmodel_file_lst = [\''+project+'.dyr\']\n')
                ini.write('\n[Tests]\nTest00_FS = [\'FS\', \'20\']\nTest01_LVRT_LEGACY = [\'VOLT\',\'DATAs\\ERCOT_Legacy_LVRT.xlsx\']\n')
                ini.write('Test02_LVRT_VOLT_DIP = [\'VOLT\',\'DATAs\\ERCOT-245_2800_Piecewise_LVRT_(PV-BESS)_10secSpacing_r2.xlsx\']\n')
                ini.write('Test03_HVRT_PREFERED = [\'VOLT\',\'DATAs\\ERCOT-245_2800_HVRT.xlsx\']\n')
                ini.write('Test04_HVRT_LEGACY = [\'VOLT\',\'DATAs\\ERCOT_Legacy_HVRT.xlsx\']\n\n[Settings]\nPlot_Flag = 1')

In [ ]:
"""This section defines a function to run the vrt given the project number and name."""

def run_vrt(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    import os
    vrt_path = 'D:/DMView 3.3.1/'
    os.chdir(vrt_path)

    initials = ['_lag','_lead']

    if has_ess(project):    # Check if there is storage
        scenarios = ['_dis','_ch']
    if has_non_ess(project):  # Check if there is solar or wind
        scenarios = ['']
    if has_ess(project) and has_non_ess(project): # Check if it's SLF or hybrid project
        scenarios = ['','_dis','_ch','_cmb']
        # scenarios = ['_dis','_ch']

    for scenario in scenarios:
        for initial in initials:
            filename = vrt_path + project + scenario + initial + '.ini'
            os.system('python dmview.py "'+filename+'"')
    print('Finish the VRT of '+project+', please go to the RESULTs folder to check if run successfully.\n')


In [ ]:
"""This section defines a function to plot the VRT results and output to a Word file."""

def plot_vrt(project, poi_chnl, ness_chnl=None, ess_chnl= None,  ness_cmb_chnl=None, ess_cmb_chnl=None, poi_cmb_chnl=None):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""
    """Input the solar or wind, ESS channels to be plotted (list) e.g. [3,5,24] """
    """Input the POI channels to be plotted (list) e.g. [4,6,28] """
    """Input the ESS channels in the combined case to be plotted (list) e.g. [3,5,24] """
    """Input the Non ESS channels in the combined case to be plotted (list) e.g. [3,5,24] """

    import sys, os
    # Set PSSE and psspy
    sys_path_PSSE = r'C:\Program Files\PTI\PSSE35\35.5\PSSPY39'  # or where else you find the psspy.pyc
    os_path_PSSE = r'C:\Program Files\PTI\PSSE35\35.5\PSSBIN'  # or where else you find the psse.exe
    sys.path.append(sys_path_PSSE)
    sys.path.append(os_path_PSSE)

    import psse35
    import psspy
    import dyntools
    import matplotlib.pyplot as plt
    import glob
    from docx import Document
    from docx.shared import Inches
    from io import BytesIO

    project_path = 'D:/ERCOT/AEP Dynamic/MQT/Undone/'
    os.chdir(project_path+project+'/')

    all_tests = ['TEST00_FS_FS','TEST01_LVRT_LEGACY_VOLT','TEST02_LVRT_VOLT_DIP_VOLT','TEST03_HVRT_PREFERED_VOLT','Test04_HVRT_LEGACY_VOLT']
    tests = all_tests[1:4]  # normal condition
    # tests = all_tests[1:]   # not pass prefered hvrt
    # tests = [all_tests[0]]    # not pass flat start

    initials = ['_lag','_lead']
    scenarios = ['']

    if has_ess(project):    # Check if there is storage
        scenarios = ['_dis','_ch']
    if has_solar(project):  # If there is solar
        type = 'Solar'
    if has_wind(project):  # If there is wind
        type = 'Wind'
    if is_slf(project): # Check if it's SLF or hybrid project
        scenarios = ['','_dis','_ch','_cmb']

    # Create document for word
    document = Document()
    # word title
    document.add_heading('VRT Report for '+project,0)

    for scenario in scenarios:
        if scenario == '_dis':
            subtitle1 = 'ESS Discharging'       # storage only
            channels_plot = [poi_chnl,ess_chnl]
            if is_slf(project):                 # slf or hybrid
                subtitle1 = type+' Off, '+subtitle1
        elif scenario == '_ch':
            subtitle1 = 'ESS Charging'
            channels_plot = [poi_chnl,ess_chnl]
            if is_slf(project):
                subtitle1 = type+' Off, '+subtitle1
        elif scenario == '_cmb':                # slf or hybrid
            subtitle1 = type+' On (50%), ESS Discharging (50%)'
            channels_plot = [poi_cmb_chnl,ness_cmb_chnl,ess_cmb_chnl]   # all channels for poi, ess, and ness
        else:
            subtitle1 = ''
            channels_plot = [poi_chnl,ness_chnl]
            if is_slf(project):
                subtitle1 = 'ESS Off, '+type

        for initial in initials:
            if initial == '_lag':
                subtitle2 = 'Lagging'
            else:
                subtitle2 = 'Leading'

            name = project.upper() + scenario + initial
            outpath = 'D:\DMView 3.3.1\RESULTs\\'+name

            document.add_heading(subtitle1 +' '+subtitle2+':')  # subtitles

            for test in tests:
                # Get the output file
                outlist = glob.glob(outpath+'\\'+test+'\*.out')
                if outlist == []:   # no case built and no output file
                    print(subtitle1 +' '+subtitle2+' case can not be built. CANNOT achieve the specified POI_PF.\n')
                    document.add_paragraph('Case can not be built, because the specified POI power factor can not be achieved. Please modify the model.')
                    break
                else:
                    out = outlist[0]

                outfile = dyntools.CHNF(out)    # load the output file

                short_title, chanid_dict, chandata_dict = outfile.get_data()    # Subtrack the information from the output file

                # Plot VRT result
                for channels in channels_plot:  # channels to plot
                    if channels == None:        # no need to plot that 3 channels
                        break
                    else:
                        p_num, q_num, v_num = channels  # get the active power, reactive power, and voltage channel numbers
                        time = chandata_dict['time']    # get the time index
                        
                        # Add titles to each plot
                        if test == 'TEST00_FS_FS':
                            title = 'Flat start result at '+chanid_dict[p_num][5:]
                        elif test == 'TEST01_LVRT_LEGACY_VOLT':
                            title = 'LVRT legacy result at '+chanid_dict[p_num][5:]
                        elif test == 'TEST02_LVRT_VOLT_DIP_VOLT':
                            title = 'LVRT voltage dip result at '+chanid_dict[p_num][5:]
                        elif test == 'TEST03_HVRT_PREFERED_VOLT' or test == 'TEST00_FS_FS':
                            title = 'HVRT prefered result at '+chanid_dict[p_num][5:]
                        elif test == 'TEST04_HVRT_LEGACY_VOLT':
                            title = 'HVRT legacy result at '+chanid_dict[p_num][5:]
                        else:
                            title = test+' at '+chanid_dict[p_num][5:]

                        labels = ['Real Power','Reactive Power','Voltage']

                        # Create a plot with two subplot up and down
                        fig, (ax1, ax2) = plt.subplots(2, sharex=True)
                        fig.suptitle(title)

                        # Plot the power in upper subplot
                        if channels == poi_chnl or channels == poi_cmb_chnl:
                            ax1.set_ylabel('Real and Reactive Power (MVA)') # use MVA if plot the POI result
                        else:
                            ax1.set_ylabel('Real and Reactive Power (pu)')  # use pu for terminal result
                        ax1.plot(time, chandata_dict[p_num], color='tab:green',)
                        ax1.plot(time, chandata_dict[q_num], color='tab:red',)
                        ax1.grid(linestyle = '--')

                        # Plot the voltage in lower subplot
                        ax2.set_xlabel('Time (seconds)')
                        ax2.set_ylabel('Voltage (pu)')
                        ax2.plot(time, chandata_dict[v_num], color='tab:blue',)
                        ax2.grid(linestyle = '--')

                        fig.tight_layout()  # otherwise the right y-label is slightly clipped

                        # set the x axis limit according to the test
                        if test == 'TEST02_LVRT_VOLT_DIP_VOLT':
                            plt.xlim(0,60)
                        elif test == 'TEST03_HVRT_PREFERED_VOLT' or test == 'TEST00_FS_FS':
                            plt.xlim(0,20)
                        else:
                            plt.xlim(0,25)

                        # Create legend for the whole plot
                        fig.legend([ax1, ax2], labels=labels ,loc = 'center right')

                        # Save the plot in IO
                        memfile = BytesIO()
                        plt.savefig(memfile)
                        # plt.show()
                        plt.close()
                        # Add the plot from IO to the document
                        document.add_picture(memfile, width=Inches(5))
                        memfile.close()
    # Save the document
    try:            
        document.save(project+'.docx')
    except PermissionError:
        # The document is opened in Word.
        print('Please close '+project+'.docx in Word and run the cell again.')

In [ ]:
"""This section runs the VRT."""
project = '27INR0152 New Shepard Solar'
create_ini(project)
run_vrt(project)

In [ ]:
"""This section plots the VRT results."""
# Combined case plot
# plot_vrt(project, poi_chnl=[29,30,28], ness_chnl=[3,5,22], ess_chnl= [3,5,22],  ness_cmb_chnl=[4,7,31], ess_cmb_chnl=[5,8,33], poi_cmb_chnl=[40,41,39])

# simple case plot
plot_vrt(project, poi_chnl=[29,30,28], ess_chnl= [3,5,24])